In [ ]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing

In [ ]:
!pip install kaggle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#download link https://www.kaggle.com/datasets/kausr25/chatterbotenglish
!unzip /content/archive.zip -d /content/chatbot_nlp/data/

Archive:  /content/archive.zip
  inflating: /content/chatbot_nlp/data/ai.yml  
  inflating: /content/chatbot_nlp/data/botprofile.yml  
  inflating: /content/chatbot_nlp/data/computers.yml  
  inflating: /content/chatbot_nlp/data/emotion.yml  
  inflating: /content/chatbot_nlp/data/food.yml  
  inflating: /content/chatbot_nlp/data/gossip.yml  
  inflating: /content/chatbot_nlp/data/greetings.yml  
  inflating: /content/chatbot_nlp/data/health.yml  
  inflating: /content/chatbot_nlp/data/history.yml  
  inflating: /content/chatbot_nlp/data/humor.yml  
  inflating: /content/chatbot_nlp/data/literature.yml  
  inflating: /content/chatbot_nlp/data/money.yml  
  inflating: /content/chatbot_nlp/data/movies.yml  
  inflating: /content/chatbot_nlp/data/politics.yml  
  inflating: /content/chatbot_nlp/data/psychology.yml  
  inflating: /content/chatbot_nlp/data/science.yml  
  inflating: /content/chatbot_nlp/data/sports.yml  
  inflating: /content/chatbot_nlp/data/trivia.yml  


In [ ]:
from tensorflow.keras import preprocessing, utils
import os
import yaml

In [ ]:
dir_path = '/content/chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)
questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1894


In [ ]:
from gensim.models import Word2Vec
import re

In [ ]:
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary

In [ ]:
#encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

(564, 22) 22


In [ ]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(564, 74) 74


In [ ]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(564, 74, 1894)


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 22)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 74)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 22, 200)      378800      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 74, 200)      378800      ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save( 'model.h5' )

Epoch 1/150
12/12 [==============================] - 14s 545ms/step - loss: 1.2938
Epoch 2/150
12/12 [==============================] - 6s 536ms/step - loss: 1.1170
Epoch 3/150
12/12 [==============================] - 6s 531ms/step - loss: 1.0936
Epoch 4/150
12/12 [==============================] - 6s 525ms/step - loss: 1.0731
Epoch 5/150
12/12 [==============================] - 6s 530ms/step - loss: 1.0533
Epoch 6/150
12/12 [==============================] - 6s 532ms/step - loss: 1.0356
Epoch 7/150
12/12 [==============================] - 6s 531ms/step - loss: 1.0215
Epoch 8/150
12/12 [==============================] - 6s 530ms/step - loss: 1.0067
Epoch 9/150
12/12 [==============================] - 6s 525ms/step - loss: 0.9943
Epoch 10/150
12/12 [==============================] - 6s 529ms/step - loss: 0.9783
Epoch 11/150
12/12 [==============================] - 6s 527ms/step - loss: 0.9648
Epoch 12/150
12/12 [==============================] - 6s 526ms/step - loss: 0.9495
Epoch 13/150

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

print("Hello, I am Bot, Chat Bot. How can I help you?")
for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'YOU : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "BOT: " + decoded_translation )

Hello, I am Bot, Chat Bot. How can I help you?
YOU : hello
BOT:  hi end
YOU : how are you
BOT:  i'm sure i am not a net end
YOU : what is computer
BOT:  a game with tall players end
YOU : how old are you
BOT:  i'm sure i think in your artificial intelligence end
YOU : what is your age
BOT:  quite young but a million times smarter than you end
YOU : who made you
BOT:  my favorite movie end
YOU : who is spiderman
BOT:  a comic book story made into a movie end
YOU : what is apple
BOT:  an established system of political administration by which a nation state district etc is governed end
YOU : which os is the best
BOT:  in course we say end
YOU : end
BOT:  hello end
